# Workload rtapp + Idle States Residency Analysis

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

2018-12-12 17:20:51,317 INFO    : root         : Using LISA logging configuration:
2018-12-12 17:20:51,319 INFO    : root         :   /home/leoy/Work2/Develop/tools/lisa/logging.conf


In [2]:
%matplotlib inline

import os

# Support to access the remote target
from env import TestEnv

# Support to access cpuidle information from the target
from devlib import *

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Periodic

# Support for trace events analysis
from trace import Trace
from trace import ResidencyTime, ResidencyData

# DataFrame support
import pandas as pd
from pandas import DataFrame

# Trappy (plots) support
from trappy import ILinePlot
from trappy.stats.grammar import Parser

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

import json

# Hikey960 Platform Configurations

In [3]:
# Setup target configuration
hikey960_conf = {
    "platform"     : 'linux',
    "board"        : 'hikey',
    "host"         : "192.168.2.24",
    "username"     : "root",
    "password"     : "root",
    "results_dir"  : "cpuidle_post_opt",
    "modules"      : [ 'cpufreq', 'cpuidle' ],
    "ftrace"       : {
        "events" : [ 
            "cpu_idle",
            "cpu_capacity",
            "cpu_frequency",
            "sched_switch",
            "sched_load_avg_cpu",
            "cpuidle_interval_stats",
            "cpuidle_state_idx",
            "cpuidle_factor",
            "irq_handler_entry",
            "ipi_entry_rcuidle",
            "softirq_raise",
            "sched_migrate_task",
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task"            
        ],
        "buffsize" : 50 * 1024,
    },
    "tools"        : [ 'trace-cmd', 'taskset', 'rt-app', 'sysbench' ],
    "rtapp-calib"  : {
        "0": 301, "1": 301, "2": 301, "3": 301, "4": 155, "5": 155, "6": 155, "7": 155,
    },  
}

# Connect with target board

In [4]:
# Support to access the remote target
from env import TestEnv

te = TestEnv(hikey960_conf)
target = te.target

2018-12-12 17:20:54,356 INFO    : TestEnv      : Using base path: /home/leoy/Work2/Develop/tools/lisa
2018-12-12 17:20:54,358 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-12-12 17:20:54,360 WARNING : TestEnv      : Wipe previous contents of the results folder:
2018-12-12 17:20:54,361 WARNING : TestEnv      :    /home/leoy/Work2/Develop/tools/lisa/results/cpuidle_post_opt
2018-12-12 17:20:54,365 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cpufreq']
2018-12-12 17:20:54,366 INFO    : TestEnv      : Connecting linux target:
2018-12-12 17:20:54,368 INFO    : TestEnv      :   username : root
2018-12-12 17:20:54,369 INFO    : TestEnv      :       host : 192.168.2.24
2018-12-12 17:20:54,370 INFO    : TestEnv      :   password : root
2018-12-12 17:20:54,372 INFO    : TestEnv      : Connection settings:
2018-12-12 17:20:54,373 INFO    : TestEnv      :    {'username': 'root', 'host': '192.168.2.24', 'password': 'root'}
2018-12-12 17:20:54,383 DEBUG 

2018-12-12 17:21:03,605 INFO    : TestEnv      :    sched_wakeup
2018-12-12 17:21:03,606 INFO    : TestEnv      :    sched_wakeup_new
2018-12-12 17:21:03,608 INFO    : TestEnv      :    softirq_raise
2018-12-12 17:21:03,609 INFO    : TestEnv      : Using configuration provided RTApp calibration
2018-12-12 17:21:03,611 INFO    : TestEnv      : Using RT-App calibration values:
2018-12-12 17:21:03,612 INFO    : TestEnv      :    {"0": 301, "1": 301, "2": 301, "3": 301, "4": 155, "5": 155, "6": 155, "7": 155}
2018-12-12 17:21:03,613 INFO    : TestEnv      : Set results folder to:
2018-12-12 17:21:03,615 INFO    : TestEnv      :    /home/leoy/Work2/Develop/tools/lisa/results/cpuidle_post_opt
2018-12-12 17:21:03,616 INFO    : TestEnv      : Experiment results available also in:
2018-12-12 17:21:03,617 INFO    : TestEnv      :    /home/leoy/Work2/Develop/tools/lisa/results_latest


In [5]:
target.cpufreq.set_all_governors('schedutil')
target.cpufreq.set_governor_tunables(
        0, 'schedutil',
        **{'rate_limit_us' : 100 }
    )

2018-12-12 17:21:03,729 DEBUG   : ssh          : (/root/devlib-target/bin/shutils cpufreq_set_all_governors schedutil); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:03,897 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:04,064 DEBUG   : ssh          : (ls -1 /sys/devices/system/cpu/cpu0/cpufreq/schedutil); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:04,228 DEBUG   : ssh          : (ls -1 /sys/devices/system/cpu/cpufreq/schedutil); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:04,396 DEBUG   : ssh          : (echo 100 > '/sys/devices/system/cpu/cpu0/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:04,562 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:04,725 DEBUG   : ssh          : (if [ -e '/sys/devices/system

In [6]:
littles_tunables = target.cpufreq.get_governor_tunables(0)
littles_tunables

2018-12-12 17:21:05,234 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:05,398 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:05,563 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:05,728 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec


{'rate_limit_us': '100'}

In [7]:
target.cpufreq.set_governor_tunables(
        4, 'schedutil',
        **{'rate_limit_us' : 100 }
    )

2018-12-12 17:21:05,915 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu4/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:06,083 DEBUG   : ssh          : (echo 100 > '/sys/devices/system/cpu/cpu4/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:06,250 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu4/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:06,418 DEBUG   : ssh          : (if [ -e '/sys/devices/system/cpu/cpu4/cpufreq/schedutil/rate_limit_us' ]; then echo 1; else echo 0; fi); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:06,584 DEBUG   : ssh          : (echo 100 > '/sys/devices/system/cpu/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:06,750 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec


In [8]:
bigs_tunables = target.cpufreq.get_governor_tunables(4)
bigs_tunables

2018-12-12 17:21:06,934 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu4/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:07,102 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu4/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:07,269 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu4/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:21:07,434 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpufreq/schedutil/rate_limit_us'); __devlib_ec=$?; echo; echo $__devlib_ec


{'rate_limit_us': '100'}

# Workload execution utility

In [9]:
def execute(te, wload, res_dir):
    
    logging.info('# Create results folder for this execution')
    !mkdir {res_dir}
    
    logging.info('# Setup FTrace')
    te.ftrace.start()

    #logging.info('## Start energy sampling')
    #te.emeter.reset()

    logging.info('### Start RTApp execution')
    wload.run(out_dir=res_dir)

    #logging.info('## Read energy consumption: %s/energy.json', res_dir)
    #nrg_report = te.emeter.report(out_dir=res_dir)

    logging.info('# Stop FTrace')
    te.ftrace.stop()

    trace_file = os.path.join(res_dir, 'trace.dat')
    logging.info('# Save FTrace: %s', trace_file)
    te.ftrace.get_trace(trace_file)

    logging.info('# Save platform description: %s/platform.json', res_dir)
    plt, plt_file = te.platform_dump(res_dir)
    
    logging.info('# Report collected data:')
    logging.info('   %s', res_dir)
    !ls -la {res_dir}
    
    return plt, plt_file, trace_file

# Workload Definition : Single task (period: 10ms, dc: 10%)

In [17]:
rtapp_name = 'example_10ms_10pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 30,  # duty cycle [%]
            duration_s     = 30, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                #'priotity' : 120,
            },
            cpus           = "4"
        ).get(),

    },
);

2018-12-12 17:23:28,159 INFO    : Workload     : Setup new workload example_10ms_10pct
2018-12-12 17:23:28,261 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:23:28,329 INFO    : Workload     : Workload duration defined by longest task
2018-12-12 17:23:28,331 INFO    : Workload     : Default policy: SCHED_OTHER
2018-12-12 17:23:28,332 INFO    : Workload     : ------------------------
2018-12-12 17:23:28,335 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER'}
2018-12-12 17:23:28,336 INFO    : Workload     :  | loops count: 1
2018-12-12 17:23:28,338 INFO    : Workload     : + phase_000001: duration 30.000000 [s] (3000 loops)
2018-12-12 17:23:28,341 INFO    : Workload     : |  period    10000 [us], duty_cycle  30 %
2018-12-12 17:23:28,343 INFO    : Workload     : |  run_time   3000 [us], sleep_time   7000 [us]
2018-12-12 17:23:28,345 INFO    : Workload     : |  CPUs affinity: 4
2018-12-12 17:23:28,346 DEBUG   

# Run the test case and generate trace data

In [18]:
res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

2018-12-12 17:23:28,887 INFO    : root         : # Create results folder for this execution


mkdir: cannot create directory ‘/home/leoy/Work2/Develop/tools/lisa/results/cpuidle_post_opt/example_10ms_10pct’: File exists


2018-12-12 17:23:29,001 INFO    : root         : # Setup FTrace
2018-12-12 17:23:29,103 DEBUG   : ssh          : (echo 51200 > '/sys/kernel/debug/tracing/buffer_size_kb'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:23:29,288 DEBUG   : ssh          : (cat '/sys/kernel/debug/tracing/buffer_size_kb'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:23:29,455 DEBUG   : ssh          : (/root/devlib-target/bin/trace-cmd reset); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:23:31,016 DEBUG   : ssh          : (/root/devlib-target/bin/trace-cmd start -e cpu_frequency -e cpu_idle -e irq_handler_entry -e sched_migrate_task -e sched_switch -e sched_wakeup -e sched_wakeup_new -e softirq_raise); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:23:32,594 DEBUG   : ssh          : (echo TRACE_MARKER_START > '/sys/kernel/debug/tracing/trace_marker'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-12-12 17:23:32,761 DEBUG   : ssh          : (/root/devlib-target/bin/shutils cpu

total 8124
drwxrwxr-x 3 leoy leoy    4096 Dec 12 17:21 .
drwxrwxr-x 3 leoy leoy    4096 Dec 12 17:21 ..
-rw-r--r-- 1 leoy leoy     639 Dec 12 17:24 example_10ms_10pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Dec 12 17:24 output.log
-rw-rw-r-- 1 leoy leoy    1514 Dec 12 17:24 platform.json
-rw-r--r-- 1 leoy leoy  372160 Dec 12 17:24 rt-app-task_p20-0.log
-rw-rw-r-- 1 leoy leoy 7921664 Dec 12 17:24 trace.dat
drwxrwxr-x 2 leoy leoy    4096 Dec 12 17:21 .trace.dat.cache


# Load Ftrace Data

In [19]:
# Time range from the analysis
(t_min, t_max) = (0, None)
    
trace = Trace(plt, trace_file, events=hikey960_conf['ftrace']['events'], window=(t_min,t_max))

/home/leoy/Work2/Develop/tools/lisa/libs/trappy/trappy/ftrace.py:201: UserWarning: Cached data is from another trace, invalidating cache.
  warnings.warn(warnstr)
2018-12-12 17:24:09,953 INFO    : Trace        : Platform clusters verified to be Frequency coherent


# Display the trace event

In [20]:
# Original TRAPpy::FTrace DataSet are still accessible by specifying the
# trace event name of interest
trace.data_frame.trace_event('sched_switch').head()

,__comm,__cpu,__line,__pid,next_comm,next_pid,next_prio,prev_comm,prev_pid,prev_prio,prev_state
Time,,,,,,,,,,,
0.000046,trace-cmd,5,2,3580,swapper/5,0,120,trace-cmd,3580,120,32
0.000307,<idle>,6,5,0,bash,3346,120,swapper/6,0,120,0
0.000803,bash,6,17,3346,kworker/6:3,3124,120,bash,3346,120,256
0.000857,kworker/dying,6,18,3124,kworker/6:0,2950,120,kworker/dying,3124,120,16
0.000887,kworker/6:0,6,19,2950,bash,3346,120,kworker/6:0,2950,120,128


In [21]:
trace.data_frame.trace_event('cpu_frequency')['frequency'] = trace.data_frame.trace_event('cpu_frequency')['frequency']/1000

In [22]:
trace.data_frame.trace_event('cpu_frequency')['__cpu']

Time
0.164377     3
0.164380     3
0.164382     3
0.164384     3
0.168958     3
0.168959     3
0.168959     3
0.168961     3
0.331105     4
0.331106     4
0.331106     4
0.331107     4
0.334145     3
0.334148     3
0.334150     3
0.334152     3
0.337297     3
0.337298     3
0.337299     3
0.337299     3
0.345224     4
0.345225     4
0.345226     4
0.345226     4
0.349835     4
0.349835     4
0.349836     4
0.349836     4
0.498024     3
0.498027     3
            ..
31.464993    3
31.464993    3
31.467591    3
31.467593    3
31.467595    3
31.467597    3
31.468666    3
31.468667    3
31.468668    3
31.468668    3
31.470671    6
31.470702    6
31.470731    6
31.470761    6
31.633464    3
31.633466    3
31.633468    3
31.633470    3
31.636861    3
31.636862    3
31.636863    3
31.636864    3
31.800404    3
31.800406    3
31.800409    3
31.800411    3
31.805206    3
31.805207    3
31.805208    3
31.805209    3
Name: __cpu, Length: 3364, dtype: int64

# Plot CPU Frequency

In [23]:
import trappy

ftrace = trace.ftrace

# These two paramatere are passed to the LinePlot call as long with the
# TRAPpy FTrace object
trappy.ILinePlot(
    
    # FTrace object
    ftrace,
    
    # Signals to be plotted
    signals= [
        'cpu_frequency:frequency',
        'sched_switch:next_pid'
    ],
    
    # Generate one plot for each value of the specified column
    #pivot='__cpu',
    
    # Generate only plots which satisfy these filters
    filters = {
        # Column
        # |     Values
        # |     |
        '__cpu' : [4, 5, 6, 7]
    },
    
    # Formatting style
    drawstyle='steps-post',
    marker = '+'

).view()

""
""
